# We test ISD on a free linear code over some ring $\mathbb Z_{p^s}$

We sample a random vector $\mathbf e\in\mathbb Z_{p^s}^n$ with weight $w$, then project to a subring (which will be a field), solve the decoding problem in the subring and see if the found solution is the projection of the solution in the starting ring

We set $w = d/2$, where $d$ is the minimum distance of the projected code

In [80]:
reset()

def gv_bound(n,k,p):
    w = 1
    while ((binomial(n,w)*(p-1)^w)<p^(n-k)):
        w +=1
    
    return w-1

In [81]:
#Set code parameters
q = 4; p = 2; k = 5; n = 20

d = gv_bound(n,k,p) #compute minimum distance
w = floor(d/2) #error vector weight

print("[q, n, k, d] = ",[q, n, k, d])
print("Doing SDP with w = ",w)

[q, n, k, d] =  [4, 20, 5, 5]
Doing SDP with w =  2


In [82]:
#Generate code
Zq = Integers(q)
A = random_matrix(Zq,n-k,k)
H = block_matrix(Zq,1,2,[identity_matrix(Zq,n-k),A])
print(H)

#Sample random error vector
e = matrix(Zq,1,n)
supp_e = Combinations(n,w).random_element() #random support
for i in supp_e:
    val = Zq.random_element()
    while val == 0:
        val = Zq.random_element()
    e[0,i] = val

#compute syndrome
s = e*H.transpose()


[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0|3 0 1 0 2]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0|2 2 2 1 1]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0|0 2 1 0 3]
[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0|1 2 1 1 1]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0|3 1 0 0 0]
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0|1 1 3 1 1]
[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0|2 3 0 2 0]
[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0|1 3 1 1 3]
[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0|1 3 1 3 1]
[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0|0 3 1 0 1]
[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0|0 1 3 3 3]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0|0 3 2 3 2]
[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0|2 3 1 2 2]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0|2 0 3 2 2]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1|1 1 2 0 0]


In [83]:
Fp = GF(2)

#Now, project the code
sub_H = H.change_ring(Fp)
sub_s = s.change_ring(Fp)

#We test Prange
solution_found = 0
while solution_found == 0:
    
    P = Permutations(n).random_element().to_matrix().change_ring(Fp) #sample permutation
    new_sub_H = sub_H*P #apply permutation
    
      
    if rank(new_sub_H[:,0:n-k])<(n-k):
        continue;
    
    
    #Do Gaussian elimination
    U = new_sub_H[:,0:n-k]^-1
    new_sub_s = sub_s*U.transpose()
    
    #Count Hamming weight
    weight_candidate = n-k-new_sub_s.list().count(0)
    if weight_candidate <= w:
        
        #invert permutation
        perm_solution = matrix(Fp,1,n)
        perm_solution[0,0:n-k] = new_sub_s
        e_prime = perm_solution*P^-1
        solution_found = 1


# Check if $\mathbf e \mod p = \mathbf e'$

In [84]:
print("Is it working? ",e.change_ring(Fp) == e_prime)
supp_e = vector(e).support()

for i in supp_e:
    print("Pos ",i,": ",e[0,i], "--> = ",e_prime[0,i])

print()
print(e)
print(e_prime)

Is it working?  True
Pos  2 :  1 --> =  1
Pos  7 :  2 --> =  0

[0 0 1 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [85]:
#Z4 = Integers(4)
#A = random_matrix(Z4,5,5)
#A = matrix(Z4,[[1, 0, 1, 3, 0],[1, 2, 3, 2, 1],[2, 1, 3, 0, 3],[1, 2, 1, 3, 2],[1, 0, 3, 2, 3]])
#print("Before")
#print(A)

def PGE_ring(H):
    
    n = H.ncols()
    r = H.nrows()
        
    ok = 1;
    i = 0;
    while (i<r)&(ok):
        #swap rows so that you have a pivot
        j = r-1-i;
        while((not (H[j,n-1-i]).is_unit())&(j>0)):
            j = j-1;
        #print("Riga per il pivot:", j)
        #if no valid element has been found, report failure
        if ((not (H[j,n-1-i]).is_unit())):
            ok = 0;
        else: #swap rows
            tmp = H[r-1-i,:];
            H[r-1-i,:] = H[j,:];
            H[j,:] = tmp;
            #scale row so that you have the pivot
            scale_coeff = H[r-1-i,n-1-i]^-1;
            H[r-1-i,:] = scale_coeff*H[r-1-i,:];
            #print("Riga scalata:")
            #print(H)
            #print("---------")
            #create zeros
            for v in range(r):
                if v!= i:
                    scale_coeff = H[r-1-v,n-1-i]
                    H[r-1-v,:] = H[r-1-v,:] - scale_coeff*H[r-1-i,:]
        i+=1;
        #print("Dopo creazione zero")
        #print(H)
        #print("fine ciclo")
    return ok, H;

#print()
#print(PGE(5,5,A)[1])

In [86]:
def swapCols(i,j,H):
    temp = H[:, i]
    H[:, i] = H[:, j]
    H[:, j] = temp
    return H

# Find the original solution

In [94]:
# We use the recovered information about e_prime
e_prime_supp = [item[1] for item in e_prime.support()]

#We test Prange
solution_found = 0
while solution_found == 0:
    
    
        
    P = Permutations(n).random_element().to_matrix().change_ring(Zq) #sample permutation
    
    # TODO: apply constraints to permutations
    
    new_H = H*P #apply permutation
    
    for i in range(len(e_prime_supp)):
        new_H = swapCols(i,e_prime_supp[i],new_H) # metto a sx le colonne per cui il supp della sol è non nullo
      
    if PGE_ring(new_H[:,0:n-k])[0]==0:
        continue;
    
    #Do Gaussian elimination
    U = new_H[:,0:n-k]^-1
    new_s = s*U.transpose()
    
    #print()
    #print(new_H)
    
    #Count Hamming weight
    weight_candidate = n-k-new_s.list().count(0)
    if weight_candidate <= w:
        
        #invert permutation
        perm_solution = matrix(Zq,1,n)
        perm_solution[0,0:n-k] = new_s
        #print("before:",perm_solution)
        for i in range(len(e_prime_supp)):
            swapCols(i,e_prime_supp[i],perm_solution) # rigiro le entrate
        e_prime = perm_solution*P^-1
        solution_found = 1

print()
print(e_prime)



[0 0 1 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0]
